In [119]:
import time, hashlib, requests, json

In [120]:
# Marvel API data
endpoint = "https://gateway.marvel.com/"
getCharsMethod = "v1/public/characters"
apiKey = "6818695382c18243570eb27db6d83eb3"
privateKey = "0ca11a0a359ac649d46d934064349266dde56d3d"
limit = "100"

In [121]:
# Open file for storing the database
charactersDb = open('characters.json', 'w')

# Clear the file to update it
charactersDb.seek(0)
charactersDb.truncate()

# Total number of characters variable
totalCharsFetched = 0
totalChars = 0

In [122]:
characters = []
# Cycle through the database
for offset in range(0, 1500, 100):
    
    # Generate md5 hash from both api keys and timestamp
    timestamp = str(int(time.time()))
    hs = hashlib.md5(timestamp.encode('utf-8') + privateKey.encode('utf-8') + apiKey.encode('utf-8')).hexdigest()
    
    # Make the request
    response = requests.get(endpoint + getCharsMethod
                            + "?" + "orderBy=name"
                            + "&" + "offset=" + str(offset) 
                            + "&" + "limit=" + limit
                            + "&" + "ts=" + timestamp 
                            + "&" + "apikey=" + apiKey
                            + "&" + "hash=" + hs)
    
    # Break if at least 1 
    # request was unsuccessful
    if(response.status_code != 200):
        print("An error " + response.status_code + " occured!")
        break
    
    # Extract the data about characters
    data = response.json()['data']
    
    # Increment total characters fetched variable
    totalCharsFetched += data['count']
    # Get the total number of characters available via API
    if(offset == 0):
        totalChars = data['total']
    
    # Write results to the file
    results = data['results']
    for character in results:
        characters.append(character)
    
    # Sleep for some time
    time.sleep(1)

# Write to the file
json.dump(characters, charactersDb, indent=2)

In [123]:
# Check if all characters are fetched
print('Total characters available via Marvel API: ' + str(totalChars))
print('Characters fetched: ' + str(totalCharsFetched))
if(totalCharsFetched == totalChars):
    print('OK! All characters has been fetched!')
else:
    print('Error! Only ' + str(totalCharsFetched) + '/' + str(totalChars) + ' have been fetched!' )
# Close database file
charactersDb.close()

Total characters available via Marvel API: 1491
Characters fetched: 1491
OK! All characters has been fetched!
